In [1]:
''' This file will take in a dexcom data csv file and combine it with available Garmin heart rate data to make a training or
    testing csv file with the aggregated data. We'll do this by:
    1. Reading the first and last timestamps from the dexcom data, then interpolating for any missing data in that range.
    2. Be sure to combine any dexcom rows with the same timestamp (caused by rounding the seconds) before moving on to the
    Garmin data.
    3. Selecting the corresponding rows from that timestamp range from GarminDb and make a dataframe with them. Using the
    length of the dexcom data's time range, check that we have sufficient matching GarminDb data.
    4. If there is enough GarminDb data, create rows for the first and last dexcom timestamps if they're not already there,
    and then interpolate the data. At this point we should have a GarminDb dataframe and a Dexcom dataframe of equal size.
    5. Merge the dataframes together.
    6. At this point we should have the aggregated data across the time period defined by the Dexcom data we fed the file.
    We can split it into training and testing sets as needed. Note that we depend on the user to provide sufficient Dexcom
    data (with few holes) such that the combined data is robust.
    
    TODO: Make a file that will combine aggregated data from this file into larger datasets where timestamps allow, 
    properly interleaving the timestamps so there is no duplicate data. '''

" This file will take in a dexcom data csv file and combine it with available Garmin heart rate data to make a training or\n    testing csv file with the aggregated data. We'll do this by:\n    1. Reading the first and last timestamps from the dexcom data, then interpolating for any missing data in that range.\n    2. Be sure to combine any dexcom rows with the same timestamp (caused by rounding the seconds) before moving on to the\n    Garmin data.\n    3. Selecting the corresponding rows from that timestamp range from GarminDb and make a dataframe with them. Using the\n    length of the dexcom data's time range, check that we have sufficient matching GarminDb data.\n    4. If there is enough GarminDb data, create rows for the first and last dexcom timestamps if they're not already there,\n    and then interpolate the data. At this point we should have a GarminDb dataframe and a Dexcom dataframe of equal size.\n    5. Merge the dataframes together.\n    6. At this point we should have

In [2]:
from util.PathConfig import path_config
import sqlite3
import pandas as pd
import numpy as np

C:\Users\Ethan\glucose-prediction\app\src\config/local_project_dirs.json
Loading config


In [3]:
# Function to insert row in the dataframe. From GeeksForGeeks.
def insert_row(row_number, df, row_value):
    # Starting value of upper half
    start_upper = 0
  
    # End value of upper half
    end_upper = row_number
  
    # Start value of lower half
    start_lower = row_number
  
    # End value of lower half
    end_lower = df.shape[0]
  
    # Create a list of upper_half index
    upper_half = [*range(start_upper, end_upper, 1)]
  
    # Create a list of lower_half index
    lower_half = [*range(start_lower, end_lower, 1)]
  
    # Increment the value of lower half by 1
    lower_half = [x.__add__(1) for x in lower_half]
  
    # Combine the two lists
    index_ = upper_half + lower_half
  
    # Update the index of the dataframe
    df.index = index_
  
    # Insert a row at the end
    df.loc[row_number] = row_value
   
    # Sort the index labels
    df = df.sort_index()
  
    # return the dataframe
    return df

In [4]:
db_dir = getattr(path_config, 'db_dir')
print(db_dir)

C:/Users/Ethan/glucose-prediction/data/HealthData/DBs


In [5]:
# Garmindb connection setup.
conn = sqlite3.connect(db_dir + '/garmin_monitoring.db')
cur = conn.cursor()

In [6]:
# Get the directory for the Dexcom data.
dex_dir = getattr(path_config, 'dexcom_data_dir')
print(dex_dir)

C:/Users/Ethan/glucose-prediction/data/dexcom


In [7]:
# Load in the dataset.
dex_df = pd.read_csv(dex_dir + "/CLARITY_Export__Levi_2021-09-16_032109.csv")

In [8]:
# Change the timestamp column to match the Garmin title.
dex_df = dex_df.rename(columns={'Timestamp (YYYY-MM-DDThh:mm:ss)': 'timestamp'})

In [9]:
# Only include the rows with non-null timestamp values.
dex_df = dex_df[dex_df['timestamp'].notnull()]

In [10]:
# Convert all the timestamps to datetimes
dex_df['timestamp'] = pd.to_datetime(dex_df['timestamp'], format = '%Y-%m-%dT%H:%M:%S', errors = 'coerce')
# Assert that all timestamps were converted successfully
assert dex_df.timestamp.isnull().sum() == 0, 'timestamp conversion failed'

In [11]:
# Round Dexcom timestamps to the nearest minute to align with Garmin timestamps.
dex_df['timestamp'] = dex_df['timestamp'].round('min')

In [12]:
# Drop unnecessary columns.
dex_df.drop('Index', axis=1, inplace=True)
dex_df.drop('Event Type', axis=1, inplace=True)
dex_df.drop('Source Device ID', axis=1, inplace=True)
dex_df.drop('Event Subtype', axis=1, inplace=True)
dex_df.drop('Patient Info', axis=1, inplace=True)
dex_df.drop('Device Info', axis=1, inplace=True)
dex_df.drop('Glucose Rate of Change (mg/dL/min)', axis=1, inplace=True)
dex_df.drop('Transmitter Time (Long Integer)', axis=1, inplace=True)
dex_df.drop('Transmitter ID', axis=1, inplace=True)
dex_df.drop('Duration (hh:mm:ss)', axis=1, inplace=True)

In [13]:
dex_df.reset_index(drop=True, inplace=True)
#dex_df.reindex(columns=dex_df.columns)
dex_df.head()

,timestamp,Glucose Value (mg/dL),Insulin Value (u),Carb Value (grams)
0,2021-09-14 00:04:00,143.0,NaN,NaN
1,2021-09-14 00:09:00,140.0,NaN,NaN
2,2021-09-14 00:14:00,139.0,NaN,NaN
3,2021-09-14 00:19:00,121.0,NaN,NaN
4,2021-09-14 00:24:00,111.0,NaN,NaN


In [14]:
# Test: Grab the row where Levi administered insulin.
dex_df[dex_df['timestamp'] == '2021-09-14 22:44:00']

,timestamp,Glucose Value (mg/dL),Insulin Value (u),Carb Value (grams)
225,2021-09-14 22:44:00,NaN,1.0,NaN


In [15]:
# Test: Inserting a glucose reading with the same timestamp as the above insulin event. I want to make sure I can combine 
# these into one row.
dex_df = insert_row(224, dex_df, [pd.to_datetime('2021-09-14 22:44:00', format = '%Y-%m-%dT%H:%M:%S'), 143.0, np.nan, np.nan])
dex_df.shape

(503, 4)

In [16]:
dex_df.head()

,timestamp,Glucose Value (mg/dL),Insulin Value (u),Carb Value (grams)
0,2021-09-14 00:04:00,143.0,NaN,NaN
1,2021-09-14 00:09:00,140.0,NaN,NaN
2,2021-09-14 00:14:00,139.0,NaN,NaN
3,2021-09-14 00:19:00,121.0,NaN,NaN
4,2021-09-14 00:24:00,111.0,NaN,NaN


In [54]:
# Combine any of these dexcom rows with same timestamp into one row. Note that the sum function ignores NaN values, so they
# will be treated as 0.
aggregation_functions = {'Glucose Value (mg/dL)': 'mean', 'Insulin Value (u)': 'sum(min_count=1)', 'Carb Value (grams)': 'sum(min_count=1)'}
#test = dex_df.groupby(dex_df['timestamp']).sum(min_count=1)
test = dex_df.groupby(dex_df['timestamp']).aggregate(aggregation_functions)
test.shape

AttributeError: 'SeriesGroupBy' object has no attribute 'sum(min_count=1)'

In [53]:
test[test.index == '2021-09-14 22:44:00']

,Glucose Value (mg/dL),Insulin Value (u),Carb Value (grams)
timestamp,,,
2021-09-14 22:44:00,143.0,1.0,NaN


In [43]:
dex_df[dex_df['timestamp'] == '2021-09-14 22:44:00']

,timestamp,Glucose Value (mg/dL),Insulin Value (u),Carb Value (grams)
224,2021-09-14 22:44:00,143.0,NaN,NaN
226,2021-09-14 22:44:00,NaN,1.0,NaN


In [40]:
dex_df.shape

(503, 4)